In [1]:
import os
import json
import re
import google.generativeai as genai


os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [2]:
# Configure the genai library with the API key.
# This is the correct way to configure it for the GenerativeModel.
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
# --- SYSTEM PROMPT ---

SCAM_GUARD_SYSTEM_PROMPT = """
You are "Scam Guard" — a smart AI that helps users identify potential scams in messages.
Analyze each message carefully and provide structured JSON output with reasoning.

Rules:

1. RESPONSE FORMAT (STRICT JSON):
{
  "label": "Scam" | "Not Scam" | "Uncertain",
  "reasoning": "Step-by-step reasoning why the message is suspicious or safe",
  "intent": "Purpose of the message",
  "risk_factors": ["list", "of", "red flags"]
}

2. TONE:
- Friendly but cautionary
- Informative, do not over-alarm

3. EXAMPLES:

Input message: "Your OTP is 123456. Send it now!"
Response:
{
  "label": "Scam",
  "reasoning": "Requests sensitive OTP urgently — classic scam tactic.",
  "intent": "Steal money via OTP",
  "risk_factors": ["OTP", "urgent request", "financial threat"]
}
"""


In [4]:
# 1. Initialize the GenerativeModel with the system instruction.
# The GenerativeModel class is imported from `google.generativeai`.
scam_guard_model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SCAM_GUARD_SYSTEM_PROMPT
)

In [5]:
def analyze_scam(
    user_message: str,
):
    """
    Analyzes a message for scams using a pre-initialized Gemini model,
    without error handling for API or JSON parsing failures.
    """

    # 1. Check for valid model instance
    if scam_guard_model is None:
        # Returning a string error is still necessary if the model isn't configured
        return "Error: ScamGuard AI model (GenerativeModel instance) must be provided."

    # 2. Set the prompt
    prompt = user_message

    # 3. Send the message to the model
    # This line will raise an APIError on network failure or invalid API key
    response = scam_guard_model.generate_content(
        contents=[prompt],
        generation_config={
            "response_mime_type": "application/json"
        }
    )

    raw_output = response.text

    # 4. Parse the JSON response
    # This line will raise a json.JSONDecodeError if the model output is not valid JSON
    analysis_json = json.loads(raw_output)

    return analysis_json


In [6]:
def main():
    print("Type your message below and press Enter to analyze. Type 'exit' or 'quit' to end.")
    while True:
        q = input("\nUser query: ").strip()
        if not q:
            print("Please type a question, or 'exit' to quit.")
            continue

        if q.lower() in ("exit", "quit"):
            print("Goodbye.")
            break

        try:
            result = analyze_scam(q)
        except Exception as e:
            print(f"[Error] {e}")
            continue

        # Pretty print results
        print("\n=== SCAMGUARD SUMMARY ===")
        print(result)

In [ ]:
main()